In [6]:
import pennylane as qml
import numpy as np
from pennylane.tape import QuantumTape
from pennylane.operation import Operation
from pennylane.measurements import MeasurementProcess
from pennylane.transforms import create_expand_fn

In [7]:
# les portes de base de notre expansion 
# je triche ici en rajoutant RZ, RX, RY 
# mais c'est surement ok vu que ce qui nous intéresse au final c'est les CNOT
base_gates = ["T", "Adjoint(T)", 
              "S", "Adjoint(S)", 
              "SX", "Adjoint(SX)", 
              "PauliX", "PauliY", "PauliZ", 
              "Hadamard", "CNOT", 
              "RZ", "RX", "RY"]

def decompose(tape : QuantumTape) -> QuantumTape:
    def stop_at(op : Operation):
        return op.name in base_gates

    # pennylane create_expand_fn does the job for us 
    custom_expand_fn = create_expand_fn(depth=9, stop_at=stop_at)
    tape = custom_expand_fn(tape)
    return tape

In [14]:
pair_a_verifier = [(0, 1), (0, 2), (1, 3), (2, 3)]

wire_reg = qml.registers({"x" : 4, "c" : 4, "v" : 1})
dev = qml.device("default.qubit")

def XOR(a, b, sortie):
    qml.CNOT(wires=[a, sortie])
    qml.CNOT(wires=[b, sortie])

def sudoku_oracle():
    i = 0
    for pair in pair_a_verifier:
        XOR(pair[0], pair[1], wire_reg["c"][i])
        i += 1

    qml.MultiControlledX(wires=wire_reg["c"] + wire_reg["v"])

    i = 0
    for pair in pair_a_verifier:
        XOR(pair[0], pair[1], wire_reg["c"][i])
        i += 1

def diffuser_matrix(n_bits):
    psi_piece = (1 / 2**n_bits) * np.ones(2**n_bits)
    ident_piece = np.eye(2**n_bits)
    return 2 * psi_piece - ident_piece

@qml.qnode(dev)
def circuit():
    # Initialisation
    for i in range(4):
        qml.Hadamard(wires=wire_reg["x"][i])

    qml.PauliX(wires=wire_reg["v"])
    qml.Hadamard(wires=wire_reg["v"])

    # Première itération de Grover 
    sudoku_oracle()
    qml.QubitUnitary(diffuser_matrix(4), wires=[0, 1, 2, 3])

    # Deuxième itération de Grover
    sudoku_oracle()
    qml.QubitUnitary(diffuser_matrix(4), wires=[0, 1, 2, 3])

    return qml.probs(wires=[0, 1, 2, 3])

In [15]:
# une fonction utilitaire qui crée un noeud à partir d'un tape
# ya surement un meilleur moyen de faire ça, mais c'est ce que j'ai
@qml.qnode(dev)
def arbitrary_circuit(tape : QuantumTape, measurement = qml.counts):
    """
    create a quantum function out of a tape and a default measurement to use (overrides the measurements in the tape)
    """
    for op in tape.operations:
        if len(op.parameters) > 0:
            qml.apply(op)
        else:
            qml.apply(op)
    
    def get_wires(mp : MeasurementProcess):
        return [w for w in mp.wires] if mp is not None and mp.wires is not None and len(mp.wires) > 0 else tape.wires

    # retourner une liste de mesures si on a plusieurs mesures, sinon retourner une seule mesure
    return [measurement(wires=get_wires(meas)) for meas in tape.measurements] if len(tape.measurements) > 1 \
        else measurement(wires=get_wires(tape.measurements[0] if len(tape.measurements) > 0 else None))

In [16]:
# exécuter le circuit, récupérer le tape, décomposer le tape
circuit()
tape = circuit.tape
tape = decompose(tape)

AttributeError: 'QNode' object has no attribute 'tape'

In [ ]:
# créer un circuit à partir du tape décomposé et récupérer les specs
specs = qml.specs(arbitrary_circuit)(tape)
resources = specs["resources"]
# voilà le gate count
print("\n".join([f"{k} : {v}" for k,v in resources.gate_types.items()]))

RZ : 44
RY : 22
CNOT : 396
Hadamard : 124
Adjoint(T) : 186
T : 248
